In [ ]:
import pandas as pd
import numpy as np
import boto
import boto3
import nltk
from nltk.tag import StanfordNERTagger
import time
from multiprocessing import Pool
st = StanfordNERTagger('/home/ec2-user/GKGPreprocessing/stanford-ner-2018-10-16/classifiers/english.all.3class.distsim.crf.ser.gz',
                       '/home/ec2-user/GKGPreprocessing/stanford-ner-2018-10-16/stanford-ner-3.9.2.jar',
                       encoding='utf-8')

In [ ]:
def getDecision(input_list):
    for item in input_list:
        if item[1] == 'ORGANIZATION':
            return "Yes"
            break   
    return "No"

In [ ]:
def ner_result(input_str):
    new_str = [w.capitalize() for w in input_str.split(' ')]
    classified_text = st.tag(new_str)
    return getDecision(classified_text)

In [ ]:
file_num = 1
start_num = 0
end_num = 10000

In [ ]:
file_path = 's3://gkgpreprocessing/'
file_name = 'df_{}.csv'.format(file_num)
output_file = 'df_{}_{}_{}.csv'.format(file_num,start_num,end_num)
new_df = pd.read_csv(file_path + file_name, index_col = 'Unnamed: 0')

In [ ]:
parsed_df = new_df.loc[start_num:end_num].copy()

In [ ]:
cleaned_name = list(parsed_df['cleaned_name'])

In [ ]:
s = time.time()
p = Pool()
result = p.map(ner_result,cleaned_name)
p.close()
p.join()
print(time.time() - s)

In [ ]:
parsed_df['original_decision'] = parsed_df['cleaned_name'].apply(ner_result)

In [ ]:
parsed_df['decision'] = pd.Series(result)

In [ ]:
parsed_df.head(10)

In [ ]:
output_df = parsed_df[parsed_df.decision == 'Yes']
output_df = output_df[['original_name','cleaned_name']]

In [ ]:
output_df.head(10)

In [ ]:
output_df.to_csv(output_file,header = False,index = False)